In [29]:
library(jsonlite)
library(tidyr)
library(ggplot2)
library(testthat)


Attaching package: ‘testthat’


The following object is masked from ‘package:tidyr’:

    matches




In [30]:
currency_code_vector<-c('AED', 'AFN', 'ALL', 'AMD', 'ANG', 'AOA', 'ARS', 'AUD', 'AWG', 'AZN',
                        'BAM', 'BBD', 'BDT', 'BGN', 'BHD', 'BIF', 'BMD', 'BND', 'BOB', 'BRL', 'BSD', 'BTC', 'BTN', 'BWP', 'BYN', 'BZD',
                        'CAD', 'CDF', 'CHF', 'CLF', 'CLP', 'CNH', 'CNY', 'COP','CRC', 'CUC', 'CUP','CVE','CZK',
                        'DJF', 'DKK', 'DOP', 'DZD',
                        'EGP', 'ERN', 'ETB', 'EUR',
                        'FJD', 'FKP',
                        'GBP', 'GEL', 'GGP', 'GHS', 'GIP', 'GMD', 'GNF', 'GTQ', 'GYD',
                        'HKD', 'HNL', 'HRK', 'HTG', 'HUF', 
                        'IDR', 'ILS', 'IMP', 'INR', 'IQD', 'IRR', 'ISK',
                        'JEP', 'JMD', 'JOD', 'JPY',
                        'KES', 'KGS', 'KHR', 'KMF', 'KPW', 'KRW', 'KWD', 'KYD', 'KZT',
                        'LAK', 'LBP', 'LKR', 'LRD', 'LSL', 'LYD',
                        'MAD', 'MDL', 'MGA', 'MKD', 'MMK', 'MNT', 'MOP', 'MRU', 'MUR', 'MVR', 'MWK', 'MXN', 'MYR', 'MZN',
                        'NAD', 'NGN', 'NIO', 'NOK', 'NPR', 'NZD', 
                        'OMR',
                        'PAB', 'PEN', 'PGK', 'PHP', 'PKR', 'PLN', 'PYG',
                        'QAR',
                        'RON', 'RSD', 'RUB', 'RWF',
                        'SAR', 'SBD', 'SCR', 'SDG', 'SEK', 'SGD', 'SHP', 'SLL', 'SOS', 'SRD', 'SSP', 'STD', 'STN', 'SVC', 'SYP', 'SZL',
                        'THB', 'TJS', 'TMT', 'TND', 'TOP', 'TRY', 'TTD', 'TWD', 'TZS',
                        'UAH', 'UGX', 'USD', 'UYU', 'UZS', 
                        'VES', 'VND', 'VUV', 
                        'WST', 
                        'XAF', 'XAG', 'XAU', 'XCD', 'XDR', 'XOF', 'XPD', 'XPF', 'XPT',
                        'YER',
                        'ZAR', 'ZMW', 'ZWL')


    A wrapper function to obtain the the latest foreign exchange reference rates of a list of currencies on a specific base  at a specific amount on daily basis from the Exchange rates API.
    
    Parameters:
    -----------
    base: Enter the three-letter currency code of your preferred base currency (e.g., base='USD').
    symbols: Enter a list of comma-separated currency codes to limit output currencies (e.g., symbols=c('USD','EUR','CZK')).
    amount: The amount to be converted (e.g., amount=1200).

    Returns:
    --------
    A dataframe with the columns `Date`, `Base`, `Rates`, and `Amount`.


In [35]:
latest_rates <- function(base="EUR", symbols='', amount=1) {
    if(base %in% currency_code_vector == FALSE){
        warning("Invalid base currency name in base!")
        return ("Error")
        }

    if(all(unlist(strsplit(symbols, ",")) %in% currency_code_vector) == FALSE){
        warning("Invalid currency names in symbols!")
        return ("Error")
        }

    if(is.numeric(amount) == FALSE){
        warning("Input a number!")
        return ("Error")
        }

    url<-paste('https://api.exchangerate.host/latest?base=',base,'&symbols=',symbols,'&amount=',amount,"&places=2",sep="")
    data <- fromJSON(url)

    if(data$success!=TRUE){
        warning("Error in connecting to the API!")
        return ("Error")
    }

    if(length(data$rates)==0){
        warning("Error in the parameter, please check!")
        return ("Error")
    }

    data <- data.frame('Date' = data$date, 'Base' = base, 'Rates' = do.call(rbind, data$rates), 'Amount' = amount)
    return (data)
}

In [36]:
a <- latest_rates('CNH','USD',1)

In [37]:
a

,Date,Base,Rates,Amount
,<chr>,<chr>,<dbl>,<dbl>
USD,2022-02-16,CNH,0.16,1


In [142]:
#source("../R/latest_rates.R")

test_that("latest rates works", {
    
    expect_equal(typeof(latest_rates('CNH','USD',1)),'list')
    expect_equal(ncol(latest_rates('CNH','USD',1)),4) 
    
    url<-'https://api.exchangerate.host/latest?base=CNH&symbols=USD&amount=1&places=2'
    data <- fromJSON(url)
    expect_equal(latest_rates('CNH','USD',1)[[1]],data$date[[1]])
    expect_equal(latest_rates('CNH','USD',1)[[2]],data$base[[1]])
    expect_equal(latest_rates('CNH','USD',1)[[3]],data$rates[[1]])
    expect_equal(latest_rates('CNH','USD',1)[[4]],1)
    
    expect_equal(typeof(latest_rates('CAD','USD,CNH',1000)),'list')
    expect_equal(ncol(latest_rates('CAD','USD,CNH',1000)), 2,4) 
    
    url<-'https://api.exchangerate.host/latest?base=CAD&symbols=USD,CNH&amount=1000&places=2'
    data <- fromJSON(url)
    expect_equal(latest_rates('CAD','USD,CNH',1000)[1,1],data$date[[1]])
    expect_equal(latest_rates('CAD','USD,CNH',1000)[2,2],data$base[[1]])
    expect_equal(latest_rates('CAD','USD,CNH',1000)[1,3],data$rates[[1]])
    expect_equal(latest_rates('CAD','USD,CNH',1000)[2,3],data$rates[[2]])
    expect_equal(latest_rates('CAD','USD,CNH',1000)[1,4],1000)      
})

test_that("latest rates throw errors and get warnings", {
    expect_warning(latest_rates('MMM','USD',1))
    expect_warning(latest_rates('CAD','USDCNH',1000))
    expect_warning(latest_rates('CAD','USDCNH','a'))
})

Test passed 🎉
Test passed 😀
